In [0]:
%scala

val storageAccount = "ybenitez01"
val container = "ybenitezcont"
val sasKey = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T13:18:10Z&st=2024-12-07T05:18:10Z&spr=https&sig=cQ9QV5TihwUaOIH6YRE7N9ueD94NN0KWBVogNkLSlXs%3D"

val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net"

dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs=Map(conf -> sasKey))

storageAccount: String = ybenitez01
container: String = ybenitezcont
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T13:18:10Z&st=2024-12-07T05:18:10Z&spr=https&sig=cQ9QV5TihwUaOIH6YRE7N9ueD94NN0KWBVogNkLSlXs%3D
conf: String = fs.azure.sas.ybenitezcont.ybenitez01.blob.core.windows.net
res0: Boolean = true

In [0]:
producto = (spark.read.parquet("/mnt/ybenitezcont/proyectofinal/capaplata/Product.parquet"))
producto.createOrReplaceTempView("s_producto")

In [0]:
categoria = (spark.read.parquet("/mnt/ybenitezcont/proyectofinal/capaplata/Categoria.parquet"))
categoria.createOrReplaceTempView("s_categoria")

In [0]:
subcategoria = (spark.read.parquet("/mnt/ybenitezcont/proyectofinal/capaplata/SubCategoria.parquet"))
subcategoria.createOrReplaceTempView("s_subcategoria")

In [0]:
%sql

DROP TABLE IF EXISTS Fact_Product;

CREATE TEMP VIEW Fact_Product
AS
SELECT P.Cod_Producto, P.Nombre AS Producto, S.Nombre AS Subcategoria, C.Nombre AS Categoria
FROM s_producto P
JOIN s_subcategoria S ON P.Cod_Subcategoria = S.Cod_Subcategoria
JOIN s_categoria C ON S.Cod_Categoria = C.Cod_Categoria; 

In [0]:
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/ybenitezcont/proyectofinal/capaoro/Fact_Producto.parquet")

In [0]:
dbutils.fs.unmount("/mnt/ybenitezcont")

/mnt/ybenitezcont has been unmounted.


True